# Initialization

In [1]:
import os

os.chdir('../')

# Imports

In [2]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch import nn, optim
import matplotlib.pyplot as plt
from torch.nn import functional as F

# Configuration

In [3]:
device = torch.device("cuda")
kwargs = {'num_workers': 1, 'pin_memory': True} 

# Parameters

In [4]:
configs = {
    "data_path": "dataset/dataset.csv",

    "signal_shape": (3000, 1),
    "label_shape": (5, 1),
    "signal_length": 3000,
    "label_length": 5,

    "latent_space": 64,
    "dropout": 0.3,

    "batch_size": 16,
    "epoches": 50,
    "learning_rate": 0.0001,

    "patience": 5,
    "monitor": "val_loss",
    "mode": "min",
}

# Dataset

In [5]:
dataset = pd.read_csv(configs['data_path'])
dataset = dataset.drop(['Unnamed: 0'], axis=1)

dataset = dataset.sample(frac=1).reset_index(drop=True)

ln = len(dataset)
train_data = dataset.iloc[:int(ln * 0.8), :]
test_data = dataset.iloc[int(ln * 0.8):int(ln * 0.9), :]
val_data = dataset.iloc[int(ln * 0.9):, :]

steps_per_epoch = len(train_data) // configs['batch_size']
steps_per_test = len(test_data) // configs['batch_size']
steps_per_val = len(val_data) // configs['batch_size']

In [18]:
class DataGenerator:
    def __init__(self, data, sig_shape, lbl_shape, batch_size, my_device):
        self.data = data
        self.sig_shape = sig_shape
        self.lbl_shape = lbl_shape
        self.batch_size = batch_size
        self.device = my_device

    def data_generation(self, idx):
        start = idx * self.batch_size

        x_init = self.data.iloc[start:start + self.batch_size, :3000]
        y_init = self.data.iloc[start:start + self.batch_size, 3000]

        x = np.empty((self.batch_size, int(self.sig_shape)))

        for i, signal in enumerate(x_init.iloc):
            signal = np.array(signal)
            x[i, :] = (signal - signal.min())/(signal.max() - signal.min())

        y_init = np.array(y_init)
        y = F.one_hot(torch.tensor(y_init, dtype=torch.int64, device=self.device), num_classes=self.lbl_shape).float()

        return torch.tensor(x, dtype=torch.float32, device=self.device), y

# Model

In [7]:
def one_hot(labels, class_size):
    targets = torch.zeros(labels.size(0), class_size)
    for i, label in enumerate(labels):
        targets[i, label] = 1
    return targets.to(device)

In [8]:
class CVAE(nn.Module):
    def __init__(self, feature_size, latent_size, class_size):
        super(CVAE, self).__init__()
        self.feature_size = feature_size
        self.class_size = class_size

        # encode
        self.fc1  = nn.Linear(feature_size + class_size, 400)
        self.fc21 = nn.Linear(400, latent_size)
        self.fc22 = nn.Linear(400, latent_size)

        # decode
        self.fc3 = nn.Linear(latent_size + class_size, 400)
        self.fc4 = nn.Linear(400, feature_size)

        self.elu = nn.ELU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x, c):
        inputs = torch.cat([x, c], 1)
        h1 = self.elu(self.fc1(inputs))
        z_mu = self.fc21(h1)
        z_var = self.fc22(h1)
        return z_mu, z_var

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z, c): 
        inputs = torch.cat([z, c], 1)
        h3 = self.elu(self.fc3(inputs))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x, c):
        mu, logvar = self.encode(x, c)
        z = self.reparameterize(mu, logvar)
        return self.decode(z, c), mu, logvar

# create a CVAE model
model = CVAE(configs['signal_length'], configs['latent_space'], configs['label_length']).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [9]:
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Train

In [20]:
train_gen = DataGenerator(train_data, 3000, 5, 16, device)

def train(epoch):
    model.train()
    train_loss = 0
    
    for i in tqdm(range(steps_per_epoch)):
        data, labels = train_gen.data_generation(i)

        recon_batch, mu, logvar = model(data, labels)
        
        optimizer.zero_grad()
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        
        train_loss += loss.detach().cpu().numpy()
        optimizer.step()
        
        if i % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(data), len(train_data),
                100. * i / len(train_data),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_data)))

In [21]:
for epoch in range(1, 50 + 1):
    train(epoch)

  1%|          | 4/509 [00:00<00:28, 17.70it/s]

Train Epoch: 1 [0/8144 (0%)]	Loss: 2097.396973


  6%|▌         | 28/509 [00:00<00:11, 42.68it/s]

Train Epoch: 1 [320/8144 (0%)]	Loss: 1905.316040


 10%|▉         | 49/509 [00:01<00:09, 47.45it/s]

...

Train Epoch: 10 [7040/8144 (5%)]	Loss: 1708.640625


 92%|█████████▏| 468/509 [00:10<00:00, 41.73it/s]

Train Epoch: 10 [7360/8144 (6%)]	Loss: 1794.216309


 94%|█████████▎| 477/509 [00:10<00:00, 43.95it/s]


KeyboardInterrupt: 